In [1]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from torchvision import transforms, models
from torchvision.models import MobileNet_V2_Weights
import torchvision.models as models

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)
from sklearn.utils.class_weight import compute_class_weight

from PIL import Image
import time
from tqdm import tqdm
import glob

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [3]:
class LabelSmoothingCrossEntropy(nn.Module):
    """Label smoothing cross entropy loss."""

    def __init__(self, smoothing: float = 0.1):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        log_prob = F.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * self.smoothing / (input.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss


In [4]:
class EarlyStopping:
    """Early stopping utility."""

    def __init__(self, patience: int = 10, min_delta: float = 0.001,
                 mode: str = 'min'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_score: float):
        if self.best_score is None:
            self.best_score = val_score
        elif self.mode == 'min':
            if val_score < self.best_score - self.min_delta:
                self.best_score = val_score
                self.counter = 0
            else:
                self.counter += 1
        else:  # mode == 'max'
            if val_score > self.best_score + self.min_delta:
                self.best_score = val_score
                self.counter = 0
            else:
                self.counter += 1

        if self.counter >= self.patience:
            self.early_stop = True


In [5]:
class MetricsCalculator:
    """Utility class for calculating classification metrics."""

    @staticmethod
    def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray,
                         y_pred_proba: np.ndarray) -> Dict[str, float]:
        """Calculate all classification metrics."""

        # Calculate confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()

        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='binary')
        recall = recall_score(y_true, y_pred, average='binary')
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        f1 = f1_score(y_true, y_pred, average='binary')
        auc_roc = roc_auc_score(y_true, y_pred_proba)

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'specificity': specificity,
            'f1': f1,
            'auc_roc': auc_roc,
            'confusion_matrix': cm
        }

    @staticmethod
    def plot_confusion_matrix(cm: np.ndarray, title: str = "Confusion Matrix"):
        """Plot confusion matrix heatmap."""
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Benign', 'Malignant'],
                   yticklabels=['Benign', 'Malignant'])
        plt.title(title)
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.show()

In [6]:
# New MobileNetV2-based classifier
class MobileNetV2Classifier(nn.Module):
    """MobileNetV2-based tumor classifier with improved architecture."""

    def __init__(self, num_classes: int = 2, dropout_rate: float = 0.5):
        super().__init__()

        # Load pretrained MobileNetV2
        self.backbone = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)

        # Get the number of features from the last layer
        in_features = self.backbone.classifier[1].in_features  # 1280

        # Replace the final classifier with a more sophisticated head
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Dropout(dropout_rate * 0.5),  # Reduced dropout for final layer
            nn.Linear(256, num_classes)
        )

        # Initialize classifier weights
        self._init_classifier_weights()

    def _init_classifier_weights(self):
        """Initialize classifier weights with Xavier initialization."""
        for m in self.backbone.classifier.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.backbone(x)

    def freeze_backbone(self):
        """Freeze backbone parameters except final classifier."""
        for name, param in self.backbone.named_parameters():
            if 'classifier' not in name:
                param.requires_grad = False

    def unfreeze_last_layers(self, num_blocks: int = 3):
        """Unfreeze last few inverted residual blocks of MobileNetV2."""
        # MobileNetV2 has features organized as Sequential modules
        # Unfreeze last num_blocks inverted residual blocks
        total_blocks = len(self.backbone.features)
        unfreeze_from = max(0, total_blocks - num_blocks)

        for i, layer in enumerate(self.backbone.features):
            if i >= unfreeze_from:
                for param in layer.parameters():
                    param.requires_grad = True

In [7]:
# Modified data transforms for better accuracy (320x320 input)
def get_improved_transforms():
    """Get improved data transforms for higher accuracy."""

    train_transforms = transforms.Compose([
        transforms.Resize((360, 360)),  # Larger resize for better crop diversity
        transforms.RandomResizedCrop(320, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=20),  # Increased rotation
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.3, contrast=0.3,
                                 saturation=0.3, hue=0.1)
        ], p=0.8),
        transforms.RandomApply([
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
        ], p=0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225]),
        # Random erasing for better generalization
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.15), ratio=(0.3, 3.3))
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((350, 350)),
        transforms.CenterCrop(320),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    return train_transforms, val_transforms


In [28]:
from torch.utils.data import Dataset
from PIL import Image

class BreakHisDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label


In [29]:
class BreakHisTrainer:
    def __init__(self, data_dir: str, batch_size=24, num_workers=4, device='cuda'):
        self.data_dir = Path(data_dir)
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.results: Dict[str, Dict] = {}
        self.train_transforms, self.val_transforms = get_improved_transforms()

    def parse_dataset(self) -> Dict[str, List]:
        print("Parsing BreakHis dataset...")
        possible_paths = [
            os.path.join(self.data_dir, 'BreaKHis_v1/BreaKHis_v1/histology_slides/breast/**/*.png'),
            os.path.join(self.data_dir, 'BreaKHis_v1/histology_slides/breast/**/*.png'),
            os.path.join(self.data_dir, 'histology_slides/breast/**/*.png'),
            os.path.join(self.data_dir, '**/*.png'),
        ]
        breast_img_paths = []
        for pattern in possible_paths:
            breast_img_paths = glob.glob(pattern, recursive=True)
            if breast_img_paths:
                print(f"Found images using pattern: {pattern}")
                break
        if not breast_img_paths:
            raise FileNotFoundError(f"No PNG files found in {self.data_dir}. Please check the dataset path.")
        print(f"Found {len(breast_img_paths)} image files")

        data_info = {'image_paths': [], 'labels': [], 'magnifications': [], 'patient_ids': []}
        for img_path in breast_img_paths:
            try:
                low = img_path.lower()
                if 'benign' in low:
                    label = 0
                elif 'malignant' in low:
                    label = 1
                else:
                    continue

                filename = os.path.basename(img_path)
                if '-40-' in filename:
                    mag = '40X'
                elif '-100-' in filename:
                    mag = '100X'
                elif '-200-' in filename:
                    mag = '200X'
                elif '-400-' in filename:
                    mag = '400X'
                else:
                    continue

                parts = filename.split('-')
                patient_id = '-'.join(parts[:3]) if len(parts) >= 3 else filename.split('.')[0]

                data_info['image_paths'].append(img_path)
                data_info['labels'].append(label)
                data_info['magnifications'].append(mag)
                data_info['patient_ids'].append(patient_id)

            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                continue

        print(f"Parsed {len(data_info['image_paths'])} images")
        print(f"Benign: {sum([l==0 for l in data_info['labels']])}, Malignant: {sum([l==1 for l in data_info['labels']])}")
        print(f"Magnifications: {set(data_info['magnifications'])}")
        print(f"Unique patients: {len(set(data_info['patient_ids']))}")

        return data_info

    def create_patient_splits(self, data_info: Dict, test_size=0.2, val_size=0.2) -> Dict[str, Dict]:
        df = pd.DataFrame(data_info)
        patient_labels = df.groupby('patient_ids')['labels'].first().reset_index()

        train_val_patients, test_patients = train_test_split(
            patient_labels['patient_ids'], test_size=test_size,
            stratify=patient_labels['labels'], random_state=42)
        train_patients, val_patients = train_test_split(
            train_val_patients,
            test_size=val_size/(1-test_size),
            stratify=patient_labels[patient_labels['patient_ids'].isin(train_val_patients)]['labels'],
            random_state=42)

        splits = {}
        for mag in ['40X','100X','200X','400X','All']:
            mask = df['magnifications'] == mag if mag != 'All' else pd.Series([True]*len(df))
            mag_df = df[mask]
            splits[mag] = {
                'train': {
                    'image_paths': mag_df[mag_df['patient_ids'].isin(train_patients)]['image_paths'].tolist(),
                    'labels': mag_df[mag_df['patient_ids'].isin(train_patients)]['labels'].tolist(),
                },
                'val': {
                    'image_paths': mag_df[mag_df['patient_ids'].isin(val_patients)]['image_paths'].tolist(),
                    'labels': mag_df[mag_df['patient_ids'].isin(val_patients)]['labels'].tolist(),
                },
                'test': {
                    'image_paths': mag_df[mag_df['patient_ids'].isin(test_patients)]['image_paths'].tolist(),
                    'labels': mag_df[mag_df['patient_ids'].isin(test_patients)]['labels'].tolist(),
                }
            }
            print(f"{mag}: train={len(splits[mag]['train']['labels'])}, "
                  f"val={len(splits[mag]['val']['labels'])}, test={len(splits[mag]['test']['labels'])}")
        return splits

    def create_data_loaders(self, splits: Dict, magnification: str) -> Dict[str, DataLoader]:
        sd = splits[magnification]
        train_ds = BreakHisDataset(sd['train']['image_paths'], sd['train']['labels'], self.train_transforms)
        val_ds = BreakHisDataset(sd['val']['image_paths'], sd['val']['labels'], self.val_transforms)
        test_ds = BreakHisDataset(sd['test']['image_paths'], sd['test']['labels'], self.val_transforms)

        train_loader = DataLoader(train_ds, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)
        val_loader = DataLoader(val_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)
        test_loader = DataLoader(test_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

        return {
            'train': train_loader,
            'val': val_loader,
            'test': test_loader
        }

    def train_epoch(self, model, loader, criterion, optimizer) -> float:
        model.train()
        total = 0.0
        for images, labs in tqdm(loader, desc="Training"):
            images, labs = images.to(self.device), labs.to(self.device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labs)
            loss.backward()
            optimizer.step()
            total += loss.item()
        return total / len(loader)

    def validate_epoch(self, model, loader, criterion) -> Tuple[float, float]:
        model.eval()
        total = 0.0
        all_p, all_l, all_proba = [], [], []
        with torch.no_grad():
            for images, labs in tqdm(loader, desc="Validation"):
                images, labs = images.to(self.device), labs.to(self.device)
                outputs = model(images)
                loss = criterion(outputs, labs)
                total += loss.item()
                probs = F.softmax(outputs, dim=1)
                preds = outputs.argmax(dim=1)
                all_p += preds.cpu().tolist()
                all_l += labs.cpu().tolist()
                all_proba += probs[:, 1].cpu().tolist()
        return total / len(loader), roc_auc_score(all_l, all_proba)

    train_magnification = train_magnification_mobilenet
    test_model_with_tta = test_model_with_tta

    def run_all_experiments(self, data_dir: str) -> pd.DataFrame:
        print("Starting MobileNetV2 experiments…")
        data_info = self.parse_dataset()
        splits = self.create_patient_splits(data_info)
        results = []
        for mag in ['40X','100X','200X','400X','All']:
            print(f"\n== Magnification: {mag}")
            m = self.train_magnification(splits, mag)
            results.append({
                'Magnification': mag,
                'accuracy': m['accuracy'],
                'precision': m['precision'],
                'recall': m['recall'],
                'specificity': m['specificity'],
                'f1': m['f1'],
                'auc_roc': m['auc_roc']
            })
            self.results[mag] = {'metrics': results[-1], 'confusion_matrix': m['confusion_matrix']}
        df = pd.DataFrame(results)
        print("\nFinal Results:\n", df.to_string(index=False))
        return df


In [30]:
# Modified train_magnification method for MobileNetV2
def train_magnification_mobilenet(self, splits: Dict, magnification: str,
                                epochs: int = 120) -> Dict[str, float]:  # Increased epochs
    """Train MobileNetV2 model for a specific magnification with improvements."""

    print(f"\n{'='*60}")
    print(f"Training on {magnification} magnification with MobileNetV2")
    print(f"{'='*60}")

    # Create data loaders
    data_loaders = self.create_data_loaders(splits, magnification)

    # Initialize model
    model = MobileNetV2Classifier(num_classes=2, dropout_rate=0.4)  # Reduced dropout
    model.to(self.device)

    # Loss function with reduced label smoothing
    criterion = LabelSmoothingCrossEntropy(smoothing=0.05)

    # Phase 1: Train classifier head with frozen backbone (longer training)
    print("\nPhase 1: Training classifier head (frozen backbone)")
    model.freeze_backbone()

    optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()),
                    lr=0.002, weight_decay=2e-4)  # Higher LR and weight decay
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs//2, eta_min=1e-6)
    early_stopping = EarlyStopping(patience=15, mode='max')  # More patience

    best_auc = 0.0
    best_model_state = None
    phase1_epochs = int(epochs * 0.4)  # 40% of epochs for phase 1

    for epoch in range(phase1_epochs):
        # Training
        train_loss = self.train_epoch(model, data_loaders['train'],
                                    criterion, optimizer)

        # Validation
        val_loss, val_auc = self.validate_epoch(model, data_loaders['val'],
                                              criterion)

        scheduler.step()
        early_stopping(-val_auc)  # Negative because we want to maximize AUC

        print(f"Epoch {epoch+1:3d}: Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val AUC: {val_auc:.4f}, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")

        # Save best model
        if val_auc > best_auc:
            best_auc = val_auc
            best_model_state = model.state_dict().copy()

        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Load best model from Phase 1
    model.load_state_dict(best_model_state)

    # Phase 2: Fine-tune with unfrozen last layers
    print("\nPhase 2: Fine-tuning with unfrozen last layers")
    model.unfreeze_last_layers(num_blocks=4)  # Unfreeze more layers

    # Lower learning rate for fine-tuning
    optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()),
                    lr=0.0002, weight_decay=1e-4)  # Lower LR
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs-phase1_epochs, eta_min=1e-7)
    early_stopping = EarlyStopping(patience=20, mode='max')  # More patience

    for epoch in range(epochs - phase1_epochs):
        # Training with gradient clipping for stability
        model.train()
        total_loss = 0.0

        for batch_idx, (images, labels) in enumerate(tqdm(data_loaders['train'], desc="Training")):
            images, labels = images.to(self.device), labels.to(self.device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()

            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            total_loss += loss.item()

        train_loss = total_loss / len(data_loaders['train'])

        # Validation
        val_loss, val_auc = self.validate_epoch(model, data_loaders['val'],
                                              criterion)

        scheduler.step()
        early_stopping(-val_auc)

        print(f"Epoch {phase1_epochs+epoch+1:3d}: Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val AUC: {val_auc:.4f}, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")

        # Save best model
        if val_auc > best_auc:
            best_auc = val_auc
            best_model_state = model.state_dict().copy()

        if early_stopping.early_stop:
            print(f"Early stopping at epoch {phase1_epochs+epoch+1}")
            break

    # Load best model and test with TTA
    model.load_state_dict(best_model_state)
    test_metrics = self.test_model_with_tta(model, data_loaders['test'])

    print(f"\nBest validation AUC: {best_auc:.4f}")
    print(f"Test metrics with TTA:")
    for metric, value in test_metrics.items():
        if metric != 'confusion_matrix':
            print(f"  {metric}: {value:.4f}")

    # Plot confusion matrix
    cm = test_metrics['confusion_matrix']
    MetricsCalculator.plot_confusion_matrix(cm, f"Confusion Matrix - {magnification} (MobileNetV2)")

    return test_metrics


In [31]:
# Test-Time Augmentation for improved accuracy
def test_model_with_tta(self, model: nn.Module, test_loader: DataLoader,
                       tta_transforms: int = 5) -> Dict[str, float]:
    """Test the model with Test-Time Augmentation for improved accuracy."""
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    # TTA transforms
    tta_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing with TTA"):
            images, labels = images.to(self.device), labels.to(self.device)

            # Original prediction
            outputs = model(images)
            probs_sum = F.softmax(outputs, dim=1)

            # TTA predictions
            for _ in range(tta_transforms):
                # Apply random augmentation
                aug_images = []
                for img in images:
                    # Convert to PIL, augment, convert back
                    img_pil = transforms.ToPILImage()(img.cpu())
                    img_aug = transforms.Compose([
                        transforms.RandomHorizontalFlip(p=0.5),
                        transforms.RandomRotation(degrees=5),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                           std=[0.229, 0.224, 0.225])
                    ])(img_pil)
                    aug_images.append(img_aug)

                aug_batch = torch.stack(aug_images).to(self.device)
                aug_outputs = model(aug_batch)
                probs_sum += F.softmax(aug_outputs, dim=1)

            # Average predictions
            avg_probs = probs_sum / (tta_transforms + 1)
            preds = avg_probs.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(avg_probs[:, 1].cpu().numpy())

    return MetricsCalculator.calculate_metrics(
        np.array(all_labels),
        np.array(all_preds),
        np.array(all_probs)
    )


In [ ]:
def main():
    """Main function to run the MobileNetV2 experiments."""

    # Download dataset using kagglehub
    import kagglehub

    # Download dataset
    path = kagglehub.dataset_download("ambarish/breakhis")
    print("Path to dataset files:", path)

    # Configuration for MobileNetV2
    DATA_DIR = path
    BATCH_SIZE = 24  # Reduced for 320x320 images
    NUM_WORKERS = 4
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

    print(f"Using device: {DEVICE}")
    print("Using MobileNetV2 backbone with improvements")

    # Initialize trainer
    trainer = BreakHisTrainer(
        data_dir=DATA_DIR,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        device=DEVICE
    )

    # Replace the train_magnification method
    trainer.train_magnification = lambda *args, **kwargs: train_magnification_mobilenet(trainer, *args, **kwargs)
    trainer.test_model_with_tta = lambda *args, **kwargs: test_model_with_tta(trainer, *args, **kwargs)

    # Run all experiments
    results_df = trainer.run_all_experiments(DATA_DIR)

    # Save results
    results_df.to_csv('breakhis_mobilenetv2_improved_results.csv', index=False)
    print("\nResults saved to 'breakhis_mobilenetv2_improved_results.csv'")

    return results_df

if __name__ == "__main__":
    main()


Path to dataset files: /kaggle/input/breakhis
Using device: cuda
Using MobileNetV2 backbone with improvements
Starting MobileNetV2 experiments…
Parsing BreakHis dataset...
Found images using pattern: /kaggle/input/breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/**/*.png
Found 7909 image files
Parsed 7909 images
Benign: 2480, Malignant: 5429
Magnifications: {'400X', '100X', '200X', '40X'}
Unique patients: 82
40X: train=1172, val=382, test=441
100X: train=1201, val=403, test=477
200X: train=1188, val=382, test=443
400X: train=1059, val=367, test=394
All: train=4620, val=1534, test=1755

== Magnification: 40X

Training on 40X magnification with MobileNetV2

Phase 1: Training classifier head (frozen backbone)


Validation: 100%|██████████| 16/16 [00:06<00:00,  2.46it/s]


Epoch   1: Train Loss: 0.7834, Val Loss: 0.8268, Val AUC: 0.7903, LR: 0.001999


Validation: 100%|██████████| 16/16 [00:06<00:00,  2.56it/s]


Epoch   2: Train Loss: 0.5332, Val Loss: 0.5554, Val AUC: 0.8114, LR: 0.001995


...

Validation: 100%|██████████| 16/16 [00:06<00:00,  2.52it/s]


Epoch  93: Train Loss: 0.2320, Val Loss: 0.5011, Val AUC: 0.8558, LR: 0.000062


Validation: 100%|██████████| 16/16 [00:05<00:00,  3.08it/s]


Epoch  94: Train Loss: 0.2315, Val Loss: 0.5137, Val AUC: 0.8634, LR: 0.000058


Training:  96%|█████████▌| 47/49 [00:30<00:01,  1.77it/s]